# Propuesta de proyecto: Uso de muestreo repetitivo para mejorar el rendimiento de una LLM en recomendación conversacional

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name_1 = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer_1 = AutoTokenizer.from_pretrained(model_name_1)
# model_1 = AutoModelForCausalLM.from_pretrained(model_name_1)

model_name_2 = "distilgpt2"
tokenizer_2 = AutoTokenizer.from_pretrained(model_name_2)
# model_2 = AutoModelForCausalLM.from_pretrained(model_name_2)

c:\Users\nicol.DESKTOP-C6FU6UA.000\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# Entrada
input_text = "Dime tu pokemon favorito"
inputs = tokenizer_1(input_text, return_tensors="pt")

# Generar texto
outputs = model_1.generate(inputs["input_ids"], max_new_tokens=100)
print(outputs)

tensor([[    1,   360,   603,  5291, 11293,  9857,  7853,  2049, 29889,    13,
            13, 29906, 29889,   376, 29925,   554,  9857,  2921, 29908,   313,
         29906, 29900, 29896, 29953,  1125,   910, 18765,   287, 16832,  3748,
         16869,   366,  4380, 21747,  9857,   297,  1855, 29899, 11526, 14354,
         29892,   322,   372, 29915, 29879,  4953,   263,  5534, 27791,   265,
         29889,    13,    13, 29941, 29889,   376,  1576,  5682,   355,   310,
         27997,  1388, 29901,  5826,   493,   310,   278, 11821, 29908,   313,
         29906, 29900, 29896, 29955,  1125,   910,  1722, 29899, 11526,  3748,
          4893,  2058,   297,   263, 13426, 29892,  1722,  3186, 29892,   322,
           372, 29915, 29879,  1063,  7213,  3368,   363,   967,   380, 27389,
         18533,   322,  5198,   414,   573,  3748,  1456, 29889]])


In [7]:
print(tokenizer_1.decode(outputs[0], skip_special_tokens=True))

Dime tu pokemon favorito.

2. "Pokemon Go" (2016): This augmented reality game lets you catch Pokemon in real-world locations, and it's become a global phenomenon.

3. "The Legend of Zelda: Breath of the Wild" (2017): This open-world game takes place in a vast, open world, and it's been praised for its stunning graphics and immersive gameplay.


In [9]:

# Entrada
input_text = "Dime tu pokemon favorito"
inputs = tokenizer_2(input_text, return_tensors="pt")

# Generar texto
outputs = model_2.generate(inputs["input_ids"], max_new_tokens=100)
print(outputs)
print(tokenizer_2.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[   35,   524, 12777, 43962,  2661, 10094,    13,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198]])
Dime tu pokemon favorito.





































































































In [12]:
%pip install accelerate

     ---------------------------------------- 0.0/354.7 kB ? eta -:--:--
     --------------------- ---------------- 204.8/354.7 kB 4.1 MB/s eta 0:00:01
     -------------------------------------- 354.7/354.7 kB 4.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: c:\Users\nicol.DESKTOP-C6FU6UA.000\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install --upgrade pip


In [12]:
from transformers import pipeline
k = 10
outputs = []
# Pipeline Initialization
pipeline = pipeline(
    "text-generation",
    model=model_name_1,
    device_map="auto",
)

# Prompt
ctx = "Eres un asistente que va directo al grano"
msg = "Entrégame una lista con las 5 canciones más famosas de Iron Maiden"
messages = [
    {
        "role": "system",
        "content": ctx,
    },
    {"role": "user", "content": msg}
]

prompt = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

for i in range(k):
    output = pipeline(
        prompt,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )
    outputs.append(output)

Device set to use cpu


In [46]:
def format_ans(ans,i=1):
    try:
        idx_1 = ans.index("1.")
        idx_2 = ans.index("2.")
        idx_3 = ans.index("3.")
        idx_4 = ans.index("4.")
        idx_5 = ans.index("5.")
        idx_c = idx_5 + ans[idx_5:].index(chr(34))
        idx_end = idx_c + ans[idx_c+1:].index(chr(34))

        answer = f"{i}. {ans[idx_1+3:idx_2-1]}, {ans[idx_2+3:idx_3-1]}, {ans[idx_3+3:idx_4-1]}, {ans[idx_4+3:idx_5-1]}, {ans[idx_5+3:idx_end+2]}\n"
        i+=1

    except:
        answer = ""

    finally:
        return answer, i
i=1
for out in outputs:
    ans, i = format_ans(out[0]["generated_text"],i)
    print(ans)

1. "The Number of the Beast", "Run to the Hills", "Flight of Icarus", "The Wicker Man", "The Trooper"

2. "The Number of the Beast", "Fear of the Dark", "The Trooper", "The Wicker Man", "Run to the Hills"

3. "The Number of the Beast", "Run to the Hills", "The Trooper", "Fear of the Dark", "Hallowed Be Thy Name"


4. "Fear of the Dark" (1984), "The Number of the Beast" (1982), "Run to the Hills" (1982), "The Trooper" (1982), "The Number of the Beast Is 666"

5. "The Number of the Beast" (1982), "Run to the Hills" (1982), "Powerslave" (1984), "Fear of the Dark" (1984), "The Trooper"

6. "Run to the Hills", "Number of the Beast", "Flight of Icarus", "The Number of the Beast", "The Trooper"

7. "Run to the Hills", "The Number of the Beast", "Hallowed Be Thy Name", "The Trooper", "Flight of Icarus"



